In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join

In [2]:
# Variabili globali

path = 'acquisizione 15_05_2023/'

In [3]:
allacqui = []

# Andiamo a prendere tutti i path dei vari file

path_acquisizioni = [f for f in listdir(path) if isfile(join(path, f))]


In [4]:
# Abbiamo usato il range siccome i csv hanno separatori diversi

for a,b in zip(path_acquisizioni,range(0,15)):
    if(b < 4 or b > 11):
        allacqui.append(pd.read_csv(path+a,sep=','))
    else:
        allacqui.append(pd.read_csv(path+a,sep='\t'))

In [5]:
# Check delle dimensioni

print(type(allacqui[4][:10].mean()))

for a in range(len(allacqui)):
    print(allacqui[a].shape)

<class 'pandas.core.series.Series'>
(120, 2)
(120, 2)
(121, 2)
(121, 2)
(14396, 2)
(14614, 2)
(14149, 2)
(14390, 2)
(119, 2)
(119, 2)
(120, 2)
(120, 2)
(0, 2)
(0, 2)
(0, 2)


In [20]:
media_mobile_ni = pd.DataFrame(columns=[])

for b in range(0,allacqui[4].shape[0],1):
    media_mobile_ni = pd.concat([media_mobile_ni,pd.DataFrame(allacqui[4][b:10+b].mean()).transpose()],ignore_index=True)

In [19]:
print(media_mobile_ni)

              Time    Voltage
0     1.684166e+15  10.004800
1     1.684166e+15  10.004800
2     1.684166e+15  10.003927
3     1.684166e+15  10.004154
4     1.684166e+15  10.003346
...            ...        ...
1435  1.684166e+15   7.876080
1436  1.684166e+15   7.870490
1437  1.684166e+15   7.874626
1438  1.684166e+15   7.882251
1439  1.684166e+15   7.879084

[1440 rows x 2 columns]


In [7]:
# Andiamo a lavorare sui dati di national instrument, ne riduciamo le dimensioni usando la media

for a in range(4,8):
    media_mobile_ni[a]

KeyError: 4